## data

In [1]:
import pandas as pd
import os
import openpyxl

In [ ]:
cr = pd.read_csv("raw_customer_review.csv")
cr

,Unnamed: 0,Dates,Ratings,Titles,Bodys
0,0,2023-02-20,5,Durable Solid excellent,Superlike
1,1,2023-02-14,5,Great quality,Fantastic
2,2,2022-12-08,4,Nice Product,Everything Nice. The Filter connection fitting...
3,3,2023-01-22,5,After 1.5year Review | Excellence at it best,Posting this review after 1.5year of usage.Thi...
4,4,2023-01-28,5,Nicee product,Washing quality is praise worthy.
...,...,...,...,...,...
2154,2154,2019-11-05,5,Less noise,Less noise
2155,2155,2019-12-18,5,Easy to use.,Nice product. Better than most popular company...
2156,2156,2020-03-10,5,Good product,It’s been 3 months since we have been using it...
2157,2157,2021-11-03,4,so far so good,Too early to give my considered opinion. Tumbl...


In [3]:
cr.drop(['Unnamed: 0'],axis=1,inplace=True)
cr

,Dates,Ratings,Titles,Bodys
0,2023-02-20,5,Durable Solid excellent,Superlike
1,2023-02-14,5,Great quality,Fantastic
2,2022-12-08,4,Nice Product,Everything Nice. The Filter connection fitting...
3,2023-01-22,5,After 1.5year Review | Excellence at it best,Posting this review after 1.5year of usage.Thi...
4,2023-01-28,5,Nicee product,Washing quality is praise worthy.
...,...,...,...,...
2154,2019-11-05,5,Less noise,Less noise
2155,2019-12-18,5,Easy to use.,Nice product. Better than most popular company...
2156,2020-03-10,5,Good product,It’s been 3 months since we have been using it...
2157,2021-11-03,4,so far so good,Too early to give my considered opinion. Tumbl...


In [4]:
cr.columns = ['date','star','title','body']
cr

,date,star,title,body
0,2023-02-20,5,Durable Solid excellent,Superlike
1,2023-02-14,5,Great quality,Fantastic
2,2022-12-08,4,Nice Product,Everything Nice. The Filter connection fitting...
3,2023-01-22,5,After 1.5year Review | Excellence at it best,Posting this review after 1.5year of usage.Thi...
4,2023-01-28,5,Nicee product,Washing quality is praise worthy.
...,...,...,...,...
2154,2019-11-05,5,Less noise,Less noise
2155,2019-12-18,5,Easy to use.,Nice product. Better than most popular company...
2156,2020-03-10,5,Good product,It’s been 3 months since we have been using it...
2157,2021-11-03,4,so far so good,Too early to give my considered opinion. Tumbl...


In [8]:
import googletrans

def translate(df):
    translator = googletrans.Translator()
    
    for i in range(len(df)):    
        try:
            result1 = translator.translate(df['body'][i],dest='en')
            #print(i,result1.text)
            df['body'][i] = result1.text
            
        except:
            continue

In [9]:
translate(cr)

C:\Users\daybi\AppData\Local\Temp\ipykernel_17860\3933437726.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['body'][i] = result1.text


In [10]:
cr

,date,star,body
0,2023-02-20,5,Durable Solid excellent Superlike
1,2023-02-14,5,Great quality Fantastic
2,2022-12-08,4,Nice Product Everything Nice. The Filter conne...
3,2023-01-22,5,After 1.5year Review | Excellence at it best P...
4,2023-01-28,5,Nicee product Washing quality is praise worthy.
...,...,...,...
2154,2019-11-05,5,Less noise Less noise
2155,2019-12-18,5,Easy to use. Nice product. Better than most po...
2156,2020-03-10,5,Good product It’s been 3 months since we have ...
2157,2021-11-03,4,so far so good Too early to give my considered...


In [11]:
cr['star'].value_counts()

5    1317
4     446
1     278
3      68
2      50
Name: star, dtype: int64

In [13]:
import pickle
import re
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords

cr['clean_text'] = pd.Series(dtype='object')
cr['stopwords_after'] = pd.Series(dtype='object')
cr['pos_tag'] = pd.Series(dtype='object')
cr['NN'] = pd.Series(dtype='object')
cr['lemmatization'] = pd.Series(dtype='object')

stops = stopwords.words('english')
stops_list = ['machine', 'www', 'http','com','ref','aspose', '\n', 'washer','washer', 'com']

for i in range(len(stops_list)):
    stops.append(stops_list[i])


def clean_text(texts): 
    corpus = []
    
    for i in tqdm(range(0, len(texts))):
        
        body = texts[i]
        #body.fillna(data[i])
        body = re.sub('[^a-zA-Z]', ' ', body) # 특수문자 제거 
        body = body.lower().split() # 대문자를 소문자로 변경, 문장을 단어 단위로 구분
        
        # 
        cr['clean_text'][i] = body
        
        no_stops = [word for word in body if not word in stops] # 불용어 제거
        cr['stopwords_after'][i] = no_stops
        
        tokens_pos = nltk.pos_tag(cr['stopwords_after'][i]) # pos tagging (품사 태깅)
        cr['pos_tag'][i] = tokens_pos
        
        NN_words = [] # 명사는 NN을 포함하고 있음을 알 수 있음(명사만 추출)
        for word, pos in tokens_pos:
            if 'NN' in pos:
                NN_words.append(word)
                cr['NN'][i] = NN_words
                
        wlem = nltk.WordNetLemmatizer() # Lemmatization(원형(lemma) 찾기) # nltk에서 제공되는 WordNetLemmatizer을 이용
        lemmatized_words = []
        for words in no_stops:
            new_word = wlem.lemmatize(words)
            lemmatized_words.append(new_word)
            cr['lemmatization'][i] = lemmatized_words
        
        corpus.append(no_stops) 
        
    return corpus


## i, x 등 길이가 2 이하인 문자 제거
error = []

for i in tqdm(range(0,len(cr['lemmatization']))):
    try:
        error.append(int(cr['lemmatization'][i]))
    except: 
        #continue
        print("It's error",i)
    
    try:
        for j in cr['lemmatization'][i]:
            if (len(j) < 3):
                cr['lemmatization'][i].remove(j)
                print(cr['lemmatization'][i])
    except:
        continue

100%|██████████| 2159/2159 [00:00<00:00, 90539.63it/s]

It's error 0
It's error 1
It's error 2
It's error 3
It's error 4
It's error 5
It's error 6
It's error 7
It's error 8
It's error 9
It's error 10
It's error 11
It's error 12
It's error 13
It's error 14
It's error 15
It's error 16
It's error 17
It's error 18
It's error 19
It's error 20
It's error 21
It's error 22
It's error 23
It's error 24
It's error 25
It's error 26
It's error 27
It's error 28
It's error 29
It's error 30
It's error 31
It's error 32
It's error 33
It's error 34
It's error 35
It's error 36
It's error 37
It's error 38
It's error 39
It's error 40
It's error 41
It's error 42
It's error 43
It's error 44
It's error 45
It's error 46
It's error 47
It's error 48
It's error 49
It's error 50
It's error 51
It's error 52
It's error 53
It's error 54
It's error 55
It's error 56
It's error 57
It's error 58
It's error 59
It's error 60
It's error 61
It's error 62
It's error 63
It's error 64
It's error 65
It's error 66
It's error 67
It's error 68
It's error 69
It's error 70
It's error 71
It

In [14]:
clean_text(cr['body'])

  0%|          | 0/2159 [00:00<?, ?it/s]C:\Users\daybi\AppData\Local\Temp\ipykernel_17860\4125447057.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr['clean_text'][i] = body
C:\Users\daybi\AppData\Local\Temp\ipykernel_17860\4125447057.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr['stopwords_after'][i] = no_stops
C:\Users\daybi\AppData\Local\Temp\ipykernel_17860\4125447057.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

[['durable', 'solid', 'excellent', 'superlike'],
 ['great', 'quality', 'fantastic'],
 ['nice',
  'product',
  'everything',
  'nice',
  'filter',
  'connection',
  'fitting',
  'standard',
  'size',
  'difficult',
  'get',
  'fitting',
  'local',
  'shops',
  'pls',
  'make',
  'standard',
  'helpful'],
 ['year',
  'review',
  'excellence',
  'best',
  'posting',
  'review',
  'year',
  'usage',
  'works',
  'like',
  'magic',
  'super',
  'easy',
  'use',
  'clean',
  'clothes',
  'low',
  'noise',
  'levels',
  'superb',
  'engineering',
  'bosch',
  'excellent',
  'user',
  'convenience'],
 ['nicee', 'product', 'washing', 'quality', 'praise', 'worthy'],
 ['best',
  'washing',
  'ever',
  'heard',
  'people',
  'saying',
  'washing',
  'usefull',
  'dirty',
  'clothes',
  'clothes',
  'get',
  'cleaned',
  'properly',
  'using',
  'washing',
  'felt',
  'misconception',
  'clothes',
  'getting',
  'super',
  'clean',
  'whole',
  'family',
  'happy',
  'bosch',
  'wm',
  'thoughts',


In [ ]:
cr.to_csv("preprocessed_customer_review.csv")